In [51]:
import os
import pandas as pd
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim
import spacy
from nltk.corpus import stopwords

In [86]:
stop_words = stopwords.words('english')
stop_words.extend(['meeting', 'january', 'mr', 'committee','federal'])

nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

c:\users\jesse\anaconda3\envs\sherry\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
c:\users\jesse\anaconda3\envs\sherry\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)


In [88]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [89]:
import pandas as pd
minutes = pd.read_csv(r'D:\lecture\NLPTA\project\code\doc_df.csv')
corpus = list(sent_to_words(minutes['content']))

In [90]:
bigram = gensim.models.Phrases(corpus, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[corpus], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [91]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):

    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [92]:
data_lemmatized[2]

[]

In [93]:
%time
corpus_no_stops = remove_stopwords(corpus)

corpus_bigrams = make_bigrams(corpus_no_stops)

data_lemmatized = lemmatization(corpus_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
id2word.filter_extremes(no_below=3, no_above=0.2)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

[['federal', 'open', 'market', 'minute', 'action', 'federal', 'open', 'market', 'hold', 'office', 'board', 'governor', 'federal', 'reserve', 'system', 'washington', 'tuesday', 'present', 'martin', 'chairman', 'haye', 'vice', 'chairman', 'brimmer_daane', 'francis', 'maisel_mitchell', 'robertson_scanlon', 'wayne', 'messrs', 'ellis', 'hickman', 'patterson', 'alternate', 'member', 'federal', 'open', 'market', 'messrs', 'bopp_clay', 'iron', 'president', 'federal', 'reserve', 'bank', 'philadelphia', 'kansas_city', 'dalla', 'respectively', 'holland', 'secretary', 'kenyon', 'assistant', 'secretary', 'broida', 'assistant', 'secretary', 'molony', 'assistant', 'secretary', 'hackley_general', 'counsel', 'brill', 'economist', 'messrs', 'baughman', 'craven', 'hersey', 'partee', 'solomon', 'associate_economist', 'holm', 'manager', 'system', 'open', 'market', 'account', 'enter', 'point', 'indicate', 'withdraw', 'point', 'indicate', 'coombs_special', 'manager', 'system', 'open', 'market', 'account', 'f

In [94]:
corpus

KeyboardInterrupt: 

In [95]:
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 3), (61, 1)]]


In [103]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [97]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

[(0, '0.023*"link" + 0.017*"forward_guidance" + 0.013*"highly_accommodative" + 0.012*"refinance" + 0.012*"threshold" + 0.012*"mbs" + 0.011*"company" + 0.010*"financing_term" + 0.009*"reinvesting_principal" + 0.009*"elizabeth_duke"'), (1, '0.046*"crease" + 0.016*"con_sistent" + 0.014*"manage" + 0.013*"swiss_national" + 0.013*"flect" + 0.012*"develop_ment" + 0.012*"mitigate" + 0.011*"main_tain" + 0.011*"operational" + 0.009*"sum"'), (2, '0.011*"district" + 0.009*"median" + 0.005*"funding" + 0.004*"cpi" + 0.004*"remove" + 0.004*"reading" + 0.004*"broadly_similar" + 0.004*"revision" + 0.004*"headline" + 0.003*"gasoline"'), (3, '0.024*"desk" + 0.013*"variable" + 0.009*"mb" + 0.009*"error" + 0.009*"tool" + 0.008*"median" + 0.008*"soma" + 0.007*"auction" + 0.007*"overnight" + 0.006*"strain"'), (4, '0.044*"subcommittee" + 0.017*"approval" + 0.013*"feasible" + 0.013*"designate" + 0.013*"repurchase" + 0.012*"fully_guaranteed" + 0.011*"consult" + 0.010*"procedure" + 0.009*"procedural_instruction"

In [98]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


In [99]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [104]:
pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.273670  0.129189       1        1  29.153404
3      0.281598  0.135746       2        1  17.623762
6     -0.257385  0.184580       3        1  16.360451
8      0.044648 -0.225325       4        1  13.938808
4     -0.041739 -0.182878       5        1  10.956771
1     -0.058964 -0.196439       6        1   4.374909
0     -0.229653  0.179633       7        1   4.369001
7     -0.041757 -0.112946       8        1   3.188751
9      0.014449  0.043924       9        1   0.030902
5      0.015133  0.044516      10        1   0.003238, topic_info=     Category        Freq                Term       Total  loglift  logprob
1207  Default  811.000000                page  811.000000  30.0000  30.0000
376   Default  760.000000        subcommittee  760.000000  29.0000  29.0000
1767  Default  631.000000                desk  631.000000  28.0000  28.0000
416   Default  308.000000              crease  308.000000  27.0000  27.0000
2929  Default  484.000000            variable  484.000000  26.0000  26.0000
109   Default  302.000000            approval  302.000000  25.0000  25.0000
190   Default  496.000000            district  496.000000  24.0000  24.0000
444   Default  291.000000               table  291.000000  23.0000  23.0000
582   Default  322.000000                   e  322.000000  22.0000  22.0000
3949  Default  608.000000              median  608.000000  21.0000  21.0000
218   Default  288.000000              figure  288.000000  20.0000  20.0000
1169  Default  223.000000            feasible  223.000000  19.0000  19.0000
36    Default  180.000000                link  180.000000  18.0000  18.0000
4342  Default  190.000000    forward_guidance  190.000000  17.0000  17.0000
182   Default  229.000000           designate  229.000000  16.0000  16.0000
922   Default  249.000000          infla_tion  249.000000  15.0000  15.0000
691   Default  229.000000        appreciation  229.000000  14.0000  14.0000
230   Default  204.000000    fully_guaranteed  204.000000  13.0000  13.0000
976   Default  227.000000            language  227.000000  12.0000  12.0000
342   Default  250.000000          repurchase  250.000000  11.0000  11.0000
4278  Default  268.000000       normalization  268.000000  10.0000  10.0000
4850  Default  234.000000        partici_pant  234.000000   9.0000   9.0000
3608  Default  352.000000                 pce  352.000000   8.0000   8.0000
2704  Default  262.000000               error  262.000000   7.0000   7.0000
3744  Default  273.000000                  mb  273.000000   6.0000   6.0000
158   Default  184.000000             consult  184.000000   5.0000   5.0000
3397  Default  308.000000                tool  308.000000   4.0000   4.0000
1312  Default  191.000000              crisis  191.000000   3.0000   3.0000
1414  Default  196.000000            duration  196.000000   2.0000   2.0000
1328  Default  199.000000          disruption  199.000000   1.0000   1.0000
...       ...         ...                 ...         ...      ...      ...
537   Topic10    0.000869            shotwell    2.452461   2.3927  -8.6376
540   Topic10    0.000869                wein    2.935005   2.2131  -8.6376
488   Topic10    0.000869              latham    3.690495   1.9841  -8.6376
65    Topic10    0.000869                bopp    6.711741   1.3860  -8.6376
473   Topic10    0.000869              duprey   11.241660   0.8702  -8.6376
57    Topic10    0.000869          vigorously   12.885781   0.7337  -8.6376
599   Topic10    0.000869  currencies_purchas    9.620475   1.0259  -8.6376
178   Topic10    0.000869             deliver   85.972816  -1.1642  -8.6376
612   Topic10    0.000869      purchased_spot    5.143236   1.6521  -8.6376
397   Topic10    0.000869             treiber   18.044121   0.3970  -8.6376
372   Topic10    0.000869                spot   94.027008  -1.2538  -8.6376
425   Topic10    0.000869               k